Graph Plot


In [ ]:
import matplotlib.pyplot as plt


def roc_plot(y_test, y_pred):
    from sklearn.metrics import roc_curve, auc
    from sklearn.preprocessing import label_binarize
    from itertools import cycle

    hand_name = {
        0: 'Nothing in hand',
        1: 'One pair',
        2: 'Two pairs',
        3: 'Three of a kind',
        4: 'Straight',
        5: 'Flush',
        6: 'Full house',
        7: 'Four of a kind',
        8: 'Straight flush',
        9: 'Royal flush',
    }

    y_test = label_binarize(y_test, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    n_classes = y_test.shape[1]

    fpr, tpr, roc_auc = dict(), dict(), dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    colors = cycle(['red', 'purple', 'green', 'orange', 'blue', 'brown', 'pink', 'gray', 'olive','cyan'])
    plt.figure(figsize=(10, 10))
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label=f'{hand_name[i]} (area = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([-.05, 1.05])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Poker Hands ROC')
    plt.legend(loc='lower right')
    plt.show(block=True)

Neural Network


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

cols = ["S1", "C1", "S2", "C2", "S3", "C3", "S4", "C4", "S5", "C5", "CLASS"]

df = pd.read_csv('./data/poker-hand-testing.data', names=cols)

x = (df.iloc[:, :10].values)
y = df['CLASS'].values

scaler = StandardScaler().fit(x)
x = scaler.transform(x)

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=.2, stratify=y)

# class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
# class_weights = {i: class_weights[i] for i in range(len(class_weights))}

y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, input_shape=(10,),
          kernel_initializer='uniform', activation='relu'))
model.add(tf.keras.layers.Dense(
    32, kernel_initializer='uniform', activation='relu'))
model.add
model.add(tf.keras.layers.Dense(
    16, kernel_initializer='uniform', activation='relu'))
model.add(tf.keras.layers.Dense(
    8, kernel_initializer='uniform', activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'],)

model.fit(x_train, y_train, epochs=10, batch_size=32, verbose=1)
y_pred = model.predict(x_test)

In [ ]:
roc_plot(y_test, y_pred)

y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

print(classification_report(y_test, y_pred, digits=3, zero_division=1))